# Project 3: Quantum Dynamics
##    Eduardo Villaseñor - 4624548
##    Eoin Horgan - 4582322



# Introduction

In this project we use numerical methods to simulate quantum systems by means of the time-dependent Schrodinguer equation:
$$-i \frac{\partial\psi}{\partial t} = H \psi $$

To use numerical methods on this equation two steps are required. Discretizing the Hamiltonian into a matrix by  discretizing  space, and the handling the time evolution (time derivative in the Schrodinger equation) by use of the Hamiltonian.

## Discretization of the Hamiltonian

The Hamiltonian is a operator that contains all the imformation about the dynamics of a quantum system
$$ H = - \frac{\hbar}{2m}\nabla^2  + \hat{V} $$

In order to model the Hamiltonian as a matrix we need to introduce a second order finite difference discretization of the domain where the problem takes place into an uniform mesh of width $h$. That is if the domain is square as $[0 \times l]^{dim}$ then $h=\frac{l}{N}$ where $dim$ equals to the dimension of the problem. Using the Taylor expansion each the second order derivatives in the
Hamiltonian can be approximated, as for example in the $x$ dimension

$$ \frac{\psi(x+h,y) + \psi(x-h,y) - 2\psi(x,y)}{h^2}   -\frac{\partial^2 \psi}{\partial x^2} = 2 \frac{h^2}{4!} \frac{\partial^4 \psi}{\partial x^4} + \cdots $$

Thus by means of this discretization a matrix $H_D$ of dimensions $N^{dim} \times N^{dim}$ is obtained as the discretization of the Hamiltonian. Where the error is of the order of $h^2$ from the Taylor expansion. Thus by taking a finer mesh we sacrifice computing time for precision in our model. Is important to note that regarless of the dimension of our problem the matrix obtained is a sparse matrix, by using *scipy.sparse* tools we can handle these matrices with much ease.

### Boundary conditions
When computing the discretization of the Hamiltonian into a matrix great care with the boundary conditions must be taken. In this work we created the code to generate both types of matrices: without boundary conditions and including Dirichlet boundary conditions. In the first case when the are no BC considered the boundaries of the domain act as infinite potential barriers making all the incindent wave functions bounce back. When Dirichlet BC are considered one must have care on the values of these boudaries as they must be included into the initial $\psi_0$. Through this work, for simplicity, we always consider the Hamiltonian matrix without the boundary, since otherwise we must add another level of complexity without any added physical advantage gained when simulating the selected quantum systems.

## Time evolution
The time evolution of the wave function is found using the Crank-Nicolson method:
$$\psi^{n+1} = \frac{1-i\Delta t H_D/2}{1+i\Delta t H_D/2} \psi^n $$

Where the in this case the time is discretized into $\Delta t$ steps. Using the package *scipy.sparse.linalg* this can easily be implemented as follows:

```python
        # Define the initial psi
        psi = psi_0 
        
        # Define the correspoding matrices
        A = (Id + 1j*H*dt/2)
        B = (Id - 1j*H*dt/2)
        
        # Evolve the system using the CN method
        while i 
            psi = sparse.linalg.spsolve(A, B.dot(psi), permc_spec='NATURAL')
```

Where due to the properties of the matrices `A` and `B` (symmetric, real and diagonally dominant) we know that a direct method using $LU$ factoriazation is going to be the most effective option. In python we use 
`spsolve`, where the `'NATURAL'` option implements an approximate minimum degree of permutations and allows for a reduced computation time.

## Simulations

The simulation relies on the files *quantum_plots.py, simulation.py* and *matrix.py*, which should be stored in a local directory. 

In [1]:
import quantum_plots as qplots
import numpy as np
import simulation as sm
from IPython.display import HTML
%load_ext autoreload
%autoreload 2

### Potential Step
For a first experiment on 1D we simulate the dynamics of a wave packet moving though a potential step

In [2]:
# Define the system parameters and domain
dim = 1
numberPoints = 256
dt = .001
dirichletBC = False
startPoint = 0
domainLength = 15

sign = -1
if dirichletBC:
    sign = 1


def potentialWell(x):
    '''A top hat potential function.'''
    mag = 200
    domain = [6, 10]
    if domain[0] < x < domain[1]:
        return mag
    else:
        return 0


potentialFunc = np.vectorize(potentialWell)


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=potentialFunc,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)

# Create the initial wave function
sim.setPsiPulse(pulse="plane", energy=500, center=2)

# System evolution and Animation
ani = qplots.animation1D(sim, psi='real', V=potentialWell, time=400)
HTML(ani.to_html5_video())

The wave gives the expected behavour, and we can se that the units are in the orders as expected. Since we are using dimensionless unts the real energy is units of $\frac{\hbar}{2m}$. As a consistency test we can also check that the wave function remains with norm equal to one.

In [3]:
norm = np.sum(sim.normPsi())
norm - 1

-8.659739592076221e-15

### Gaussian Well
Now for a more general continous potential well

In [4]:
# Define the system parameters and domain
dim = 1
numberPoints = 256
dt = .001
dirichletBC = False
startPoint = 0
domainLength = 15


def gaussian(x):
    '''A 1D Gaussian potential function'''
    mag = 200
    center = 6
    std = 1
    V = mag * np.exp(-(x-center)**2/(2*std**2))
    return V


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=gaussian,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)

# Create the initial wave function
sim.setPsiPulse(pulse="plane", energy=500, center=2)

# System evolution and Animation
ani = qplots.animation1D(sim, psi='real', V=gaussian, time=400)
HTML(ani.to_html5_video())

In [12]:
norm = np.sum(sim.normPsi())
norm - 1

-5.5511151231257827e-16

## 2D Simulations:
### Double Slit interference
Running animations in 2D can become very computationally intensive, especially for finner discretizations of the Hamiltonian. As an example of one of the results of this experiment we include this image that shows how the constructive and destructive fringes of the double slit experiment can be modeled with the tools created.

![title](results/double_slit.png)

Note: If you would like to see the potential as well, then run the local files instead, there it is possible to view both overlaid on each other.

In [5]:
# Define the system parameters and domain
dim = 2
numberPoints = 100
dt = .001
dirichletBC = False
startPoint = [0, 0]
domainLength = 2


def doubleSlit(x, y):
    '''A potential wall with two identical apertures'''
    sS = .4  # Slit separation
    sW = .05  # Slits width
    spX = .5  # X coordinate start
    spY = 1   # Y coordinate of center of slits
    bW = 0.03  # Barrier width
    mag = 20000   # Barrier potential
    if x > spX and x < spX+bW and (y < spY-sS/2. or y > spY+sS/2.):
        return mag
    if x > spX and x < spX+bW and (y > spY-sS/2.+sW and y < spY+sS/2.-sW):
        return mag
    else:
        return 0


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=doubleSlit,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)

# Create the initial wave function
sim.setPsiPulse(pulse="circular", energy=1000, center=[.1, 1],vel=[2, 0], width=.3)


# System evolution and Animation
ani = qplots.animation2D(sim, psi="norm",
                         potentialFunc='none', time=30, save=False)
HTML(ani.to_html5_video())

### Rutherford scattering
Now we simulate the famous experiment that allowed for the discovery of the nucleous of the atoms, the Rutherford scattering of particles due to a Coulomb interaction
$$ U(r) = \frac{\alpha}{r}$$
It can be seen from the video below that the vast majority of the wave travels forward unhindered.

In [6]:
# Define the system parameters and domain
dim = 2
numberPoints = 100
dt = .001
dirichletBC = False
startPoint = [0, 0]
domainLength = 2

sign = -1
if dirichletBC:
    sign = 1
allPoints = numberPoints + sign


def scattering(x, y):
    '''1/r^2 Dispersive force around a defined center '''
    center = [0.5, 1.]
    alpha = 10
    r = np.linalg.norm([x - center[0], y - center[1]])
    return alpha/r


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=scattering,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)

# Create the initial wave function
sim.setPsiPulse(pulse="circular", energy=1000, vel=[1, 0], center=[.1, 1], width=.1)


# System evolution and Animation
ani = qplots.animation2D(sim, psi="norm",
                     potentialFunc='none', save=False, time=30)
HTML(ani.to_html5_video())

### Quantum Billiards

An example of chaotic interaction.

In [7]:
# Define the system parameters and domain
dim = 2
numberPoints = 100
dt = .001
dirichletBC = False
startPoint = [0, 0]
domainLength = 1

def lorentzGas(x, y):
    center = [0.5, 0.5]
    r = np.linalg.norm([x - center[0], y - center[1]])
    if r < .2:
        return 20000
    else:
        return 0


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=lorentzGas,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)



sim.setPsiPulse(pulse="circular", energy=1000, center=[.3, .2], vel=np.random.rand(2),
                width=.1)

for i in range(50):
    sim.evolve()

# System evolution and Animation
ani = qplots.animation2D(sim, psi="norm", potentialFunc='none', save=False,
                         time=50)
HTML(ani.to_html5_video())